# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [24]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import geoviews as gv
import holoviews as hv
import json

# Import API key
from api_keys import geoapify_key

In [25]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ar rutbah,33.04,40.28,65.05,44,0,2.95,IQ,1697587954
1,1,muynoq,43.77,59.02,54.84,64,95,7.99,UZ,1697587954
2,2,edinburgh of the seven seas,-37.07,-12.31,54.86,75,100,12.41,SH,1697587701
3,3,saint-pierre,-21.34,55.48,73.08,73,0,5.75,RE,1697587954
4,4,alice springs,-23.70,133.88,72.95,14,0,11.50,AU,1697587954


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [26]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    y= 'Lat', 
    x= 'Lng', 
    geo=True, 
    size='Humidity', 
    color='City', 
    tiles='OSM', 
    frame_width= 700, 
    frame_height= 500,
    alpha=0.5,
    title= 'Humidity per City')

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [27]:
# Narrow down cities that fit criteria and drop any results with null values
humidity_condition = city_data_df['Humidity'] < 50
max_temp_condition = city_data_df['Max Temp'] < 80
filtered_city_df = city_data_df[humidity_condition & max_temp_condition]
# filtered_city_df = city_data_df[max_temp_condition]

# Drop any rows with null values
dropped_df = filtered_city_df.dropna()

# Display sample data
dropped_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ar rutbah,33.04,40.28,65.05,44,0,2.95,IQ,1697587954
4,4,alice springs,-23.70,133.88,72.95,14,0,11.50,AU,1697587954
11,11,kaitong,44.81,123.15,53.19,36,100,6.31,CN,1697587955
20,20,spearfish,44.49,-103.86,66.11,40,0,0.00,US,1697587956
45,45,nephi,39.71,-111.84,74.28,23,38,5.28,US,1697587959
...,...,...,...,...,...,...,...,...,...,...
486,486,biloela,-24.42,150.50,79.95,33,14,9.17,AU,1697588005
509,509,ararat,39.83,44.70,55.74,49,66,3.85,AM,1697588007
515,515,new norfolk,-42.78,147.06,66.70,32,45,3.24,AU,1697588008
532,532,sierra grande,-41.61,-65.36,62.69,28,2,7.83,AR,1697588010


### Step 3: Create a new DataFrame called `hotel_df`.

In [28]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = dropped_df.drop(columns=['Max Temp', 'Cloudiness', 'Wind Speed', 'Date', 'City_ID']).copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Lat,Lng,Humidity,Country,Hotel Name
0,ar rutbah,33.04,40.28,44,IQ,
4,alice springs,-23.70,133.88,14,AU,
11,kaitong,44.81,123.15,36,CN,
20,spearfish,44.49,-103.86,40,US,
45,nephi,39.71,-111.84,23,US,
...,...,...,...,...,...,...
486,biloela,-24.42,150.50,33,AU,
509,ararat,39.83,44.70,49,AM,
515,new norfolk,-42.78,147.06,32,AU,
532,sierra grande,-41.61,-65.36,28,AR,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [29]:
# Set parameters to search for a hotel
radius = 10000
categories = 'accommodations.hotel'
params = {
    'categories': categories,
    'apiKey':geoapify_key,
    'limit': 1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary and convert to a json file
    # response = requests.get(base_url, params=params, headers={'apiKey': geoapify_key})
    name_address = requests.get(base_url, params=params, headers={'apiKey': geoapify_key}).json()
    # name_address = name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
ar rutbah - nearest hotel: No hotel found
alice springs - nearest hotel: No hotel found
kaitong - nearest hotel: No hotel found
spearfish - nearest hotel: No hotel found
nephi - nearest hotel: No hotel found
idri - nearest hotel: No hotel found
blackmans bay - nearest hotel: No hotel found
bejaia - nearest hotel: No hotel found
crane - nearest hotel: No hotel found
moranbah - nearest hotel: No hotel found
san patricio - nearest hotel: No hotel found
djelfa - nearest hotel: No hotel found
aksu - nearest hotel: No hotel found
orchard homes - nearest hotel: No hotel found
port lincoln - nearest hotel: No hotel found
carnarvon - nearest hotel: No hotel found
ola - nearest hotel: No hotel found
arkalyk - nearest hotel: No hotel found
turpan - nearest hotel: No hotel found
al kharijah - nearest hotel: No hotel found
rudbar - nearest hotel: No hotel found
smara - nearest hotel: No hotel found
san rafael - nearest hotel: No hotel found
vernon - nearest hotel: No hotel fou

,City,Lat,Lng,Humidity,Country,Hotel Name
0,ar rutbah,33.04,40.28,44,IQ,No hotel found
4,alice springs,-23.70,133.88,14,AU,No hotel found
11,kaitong,44.81,123.15,36,CN,No hotel found
20,spearfish,44.49,-103.86,40,US,No hotel found
45,nephi,39.71,-111.84,23,US,No hotel found
...,...,...,...,...,...,...
486,biloela,-24.42,150.50,33,AU,No hotel found
509,ararat,39.83,44.70,49,AM,No hotel found
515,new norfolk,-42.78,147.06,32,AU,No hotel found
532,sierra grande,-41.61,-65.36,28,AR,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [43]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    y= 'Lat', 
    x= 'Lng', 
    geo=True, 
    size=10, 
    color='red', 
    tiles='EsriNatGeo', 
    frame_width= 700, 
    frame_height= 500,
    title= 'Hotels Within 10,000m of Each City',
    hover_cols=['City', 'Hotel Name', 'Country']
    )

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)